In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig
from transformers import BitsAndBytesConfig
from peft import LoraConfig
from datasets import load_dataset

MODEL_PATH="/shared/vsathia2/hf_models/relu-llama/"

In [5]:
# Load the model and tokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.eos_token = tokenizer.pad_token
# max_seq_length = 150
m = AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="cuda:0",attn_implementation="eager")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
m

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): ReLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 